In [45]:
%load_ext autoreload
%autoreload 2

import json
import re
from datetime import datetime
from gridfs import GridFS, NoFile
from pymongo import MongoClient
from bson.objectid import ObjectId
from db_interface import addMagresFile, databaseSearch, removeMagresFiles, editMagresFile
from db_schema import magresVersionSchema

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
opt_re = re.compile('Optional\(\'([a-zA-Z\-]+)\'\)')
[opt_re.match(str(k)).groups()[0] for k in magresVersionSchema._schema.keys() if opt_re.match(str(k)) != None]

['csd-num', 'csd-ref', 'doi', 'notes']

In [47]:
# MAGRES database
host = 'wigner.esc.rl.ac.uk'
#host = 'www.ccpnc.ac.uk'

client = MongoClient(host=host, port=27017)
ccpnc = client.ccpnc

In [48]:
json.dumps(datetime.now(), default=str)

'"2018-10-12 12:07:06.517413"'

In [49]:
for f in ccpnc.test.find({'x': 5}):
    print f

In [50]:
magresMdata = ccpnc.magresMetadata

In [51]:
magresIndex = ccpnc.magresIndex

In [85]:
magresIndex.find_one({'chemname': 'Eth_molecules'})['stochiometry']

[{u'n': 2, u'species': u'C'},
 {u'n': 6, u'species': u'H'},
 {u'n': 1, u'species': u'O'}]

In [84]:
for m in magresIndex.find({'$and': [{'stochiometry': [(u'C', 2), (u'H', 6), (u'O', 1)]}]}):
    print m

In [9]:
for f in magresIndex.find({}):
    print f['_id']
    print magresMdata.find({'_id': ObjectId(f['metadataID'])}).next()['version_history']

5a71e12a0c81be3fcf3ac179
[{u'date': datetime.datetime(2018, 1, 31, 15, 30, 49, 989000), u'doi': u'10.0001', u'magresFilesID': u'5a71e12a0c81be3fcf3ac177'}, {u'date': datetime.datetime(2018, 3, 8, 15, 8, 33, 557000), u'doi': u'10.00012', u'magresFilesID': u'5a71e12a0c81be3fcf3ac177'}, {u'date': datetime.datetime(2018, 3, 8, 15, 40, 20, 767000), u'doi': u'10.00012', u'magresFilesID': u'5aa159643207863732c98936'}, {u'date': datetime.datetime(2018, 3, 8, 15, 46, 56, 109000), u'doi': u'10.00015', u'magresFilesID': u'5aa15af03207863732c9893c'}, {u'date': datetime.datetime(2018, 3, 8, 16, 6, 24, 215000), u'doi': u'thing', u'magresFilesID': u'5aa15f803207863732c98953'}]
5a71e1530c81be3fcf3ac17f
[{u'date': datetime.datetime(2018, 1, 31, 15, 31, 31, 89000), u'doi': u'10.0002', u'magresFilesID': u'5a71e1530c81be3fcf3ac17d'}]
5a746e080c81be236bff2994
[{u'date': datetime.datetime(2018, 2, 2, 13, 56, 24, 292000), u'doi': u'10.0001', u'magresFilesID': u'5a746e080c81be236bff2992'}]
5aa13de33207861d13a

In [30]:
reg = re.compile('Test')
for f in ccpnc.magresIndex.find({'$and': [{'chemname': {'$regex': reg}}]}):
    print f['chemname']

TestOne
TestDOI


In [17]:
for f in magresMdata.find({'thing': 'date'}):
    print f['date']

2018-01-31 14:51:14.696000
2018-01-31 14:51:22.835000


In [25]:
magresFilesFS = GridFS(ccpnc, 'magresFilesFS')

In [5]:
gf = magresFilesFS.get(ObjectId('5a6f0c0c3207864b03da169a'))

NoFile: no file in gridfs collection Collection(Database(MongoClient(host=['wigner.esc.rl.ac.uk:27017'], document_class=dict, tz_aware=False, connect=True), u'ccpnc'), u'magresFilesFS.files') with _id ObjectId('5a6f0c0c3207864b03da169a')

In [6]:
databaseSearch([{'type': 'orcid', 'args': {'orcid': '0000-0003-4851-1144'}}])

TypeError: datetime.datetime(2018, 1, 31, 14, 42, 6, 117000) is not JSON serializable

In [48]:
ccpnc.collection_names()

[u'magresFilesFS',
 u'magresFilesFS.chunks',
 u'magresData',
 u'magresIndex',
 u'magresFilesFS.files',
 u'magresFiles',
 u'magresMetadata']

In [81]:
mfile = open('tests/ethanol.magres').read()
addMagresFile(mfile, chemname='thingNow', orcid={u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'})

1

In [25]:
def findById(fileID):
    return ccpnc.magresFiles.find({'_id': ObjectId(fileID)})

In [75]:
ff = ccpnc.magresIndex.find({"chemname": "thingNow"})
# Get the first magres file in the query
mfiles = []
for f in ff:
    print f['_id']
    print ccpnc.magresMetadata.find({'_id': ObjectId(f['metadataID'])}).next()

In [76]:
for f in ccpnc.magresIndex.find({'_id': ObjectId('5a6f0c0c3207864b03da168c')}):
    print f

In [79]:
removeMagresFiles('5a6f0caf3207864b03da169a')

RuntimeError: No entry with that id found

In [38]:
ff = ccpnc.magresMetadata.find({"chemname": "thingNow"})

for f in ff:
    print f['version_history']

[{u'magresFilesID': ObjectId('5a6b61930c81be53f30a4470')}]


In [19]:
ff = ccpnc.magresData.find({"orcid.path": "0000-0003-4851-1144"})
for f in ff:
    print f['orcid']

{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}


In [6]:
from db_schema import magresDataSchema

In [12]:
d = f.copy()
del d['_id']
del d['magresFilesID']

d['csd-ref'] = {
    'refcode': 'C6H6OH',
    'digits': '01'
}

magresDataSchema.validate(d)

{u'chemname': u'Ethanol (additional file)',
 'csd-ref': {'digits': '01', 'refcode': 'C6H6OH'},
 u'doi': u'10.0001',
 u'orcid': {u'host': u'orcid.org',
  u'path': u'0000-0003-4851-1144',
  u'uri': u'http://orcid.org/0000-0003-4851-1144'},
 u'values': [{u'iso': [29.559937639131203,
    30.22618664854139,
    30.072254456072027,
    26.953990818833876,
    27.373946759128433,
    31.888119371185127],
   u'species': u'H'},
  {u'iso': [156.1229153499832, 109.35753044488325], u'species': u'C'},
  {u'iso': [267.0122765992025], u'species': u'O'}]}

In [8]:
import re

In [9]:
myre = re.compile('[0-9]{2}\Z')

In [10]:
myre.match('34')